# Creating input dataset for MetSim

Alright, now that you've gotten a feel for how to work with MetSim you may be wondering how to bring in new data.
In this portion of the tutorial we will put together a new MetSim setup for Reynolds Creek.
This is an experimental watershed in southwestern Idaho.
We will be looking at a single location for a year.
The data was downloaded in csv format, which we will transform into NetCDF inputs.
As usual, we begin with some standard imports.

In [ ]:
%pylab inline
import cartopy
import geoviews as gv
import geopandas as gpd
import holoviews as hv
import pandas as pd
import xarray as xr
from metsim import MetSim

pylab.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 96
hv.notebook_extension('bokeh')

In [ ]:
(gv.tile_sources.StamenTerrainRetina
 * gv.Points([(-116.51, 43.17)]).opts(style=dict(size=16, color='red'))
 * gv.Points([(-122.3321, 49.9),(-122.8443, 47.2529), (-122.5, 42.0)]).opts(style=dict(size=0))
).opts(width=900, height=600)

# Put together the required meteorological data
We've gathered some data form the Reynolds Creek site that we will use as input.
There are two sets of CSVs with precipitation and temperature data, respectively.
We data for 2009 and 2010, and will be generating MetSim input for 2010.
To do this we must first convert it into an `xarray` dataset.
Before doing that though, let's just open up one of the temperature files and see what we're working with.

In [ ]:
%%bash
head -n 10 ./reynolds_creek_data/daily/temp_2010.csv
echo ""

Here we see that we have several pieces of information to weed through.
Luckily, pandas makes this quite easy. We can load this into a dataframe and easily select out the columns that we want.
As we can see from the header, we are going to be interested in the daily values `TMAX.D-1` and `TMIN.D-1`.
Similarly, the column we are interested in, in the precipitation data is `PREC.I-1`.
So, let's load things up.

## Create the dataset with the relevant dimensions.

In [ ]:
dates = pd.date_range('1/1/2010', '12/31/2010')
shape = (len(dates), 1, 1, )
dims = ('time', 'lat', 'lon', )

# We are running only one site, at these coordinates
lats = [43.17]
lons = [-116.51]
elev = 1706.90 # meters
coords = {'time': dates, 'lat': lats, 'lon': lons}

# Create the initial met data input data structure
met_data = xr.Dataset(coords=coords)
met_data

## Create the actual data arrays to put the data into.

In [ ]:
for varname in ['prec', 't_min', 't_max']:
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname)

## Read in the data and put it into the dataset

In [ ]:
# Add the precipitation data
df = pd.read_csv("./reynolds_creek_data/daily/precip_2010.csv", skiprows=[0,1])
# Convert to mm
prec_vals = df['PREC.I-1 (in) '].diff().values[1:] * 25.4
met_data['prec'].values[:, 0, 0] = prec_vals

# And now temp data
df = pd.read_csv("./reynolds_creek_data/daily/temp_2010.csv", skiprows=[0,1])
tmin_vals = df['TMIN.D-1 (degC) '].values[1:]
tmax_vals = df['TMAX.D-1 (degC) '].values[1:]
met_data['t_min'].values[:, 0, 0] = tmin_vals
met_data['t_max'].values[:, 0, 0] = tmax_vals

# This is what we have now
met_data.to_netcdf('./input/rc_forcing.nc')
met_data

# Put together the required domain 

In [ ]:
# We form the domain in a similar fashion
# First, by creating the data structure
coords = {'lat': lats, 'lon': lons}
domain = xr.Dataset(coords=coords)
domain['elev'] = xr.DataArray(data=np.full((1,1,), np.nan),
                          coords=coords,
                          dims=('lat', 'lon', ))
domain['mask'] = xr.DataArray(data=np.full((1,1,), np.nan),
                          coords=coords,
                          dims=('lat', 'lon', ))

# Add the data
domain['elev'][0, 0] = elev
domain['mask'][0, 0] = 1
domain.to_netcdf('./input/rc_domain.nc')
domain

### Put together the required state

In [ ]:
# Finally, we create the state file - the dates are 90 days prior to 
# the MetSim run dates - as usual, create an empty data structure to
# read the data into
dates = pd.date_range('10/3/2009', '12/31/2009')
shape = (len(dates), 1, 1, )
dims = ('time', 'lat', 'lon', )
lats = [43.17]
lons = [-116.51]
elev = 1706.90 # meters
coords = {'time': dates, 'lat': lats, 'lon': lons}
state = xr.Dataset(coords=coords)
for varname in ['prec', 't_min', 't_max']:
    state[varname] = xr.DataArray(data=np.full(shape, np.nan),
                               coords=coords, dims=dims,
                               name=varname)

# Do precip data
df = pd.read_csv("./reynolds_creek_data/daily/precip_2009.csv", skiprows=[0,1])
prec_vals = df['PREC.I-1 (in) '].diff().values[-90:] * 25.4
state['prec'].values[:, 0, 0] = prec_vals

# And now temp data
df = pd.read_csv("./reynolds_creek_data/daily/temp_2009.csv", skiprows=[0,1])
tmin_vals = df['TMIN.D-1 (degC) '].values[-90:]
tmax_vals = df['TMAX.D-1 (degC) '].values[-90:]
state['t_min'].values[:, 0, 0] = tmin_vals
state['t_max'].values[:, 0, 0] = tmax_vals
state.to_netcdf('./input/rc_state.nc')
state

## Registering parameters and building the driver
Now that we've built all of the input files we need, we can run MetSim with our new setup.
Again, we build a simple configuration and run just as we did in the previous notebook.

In [ ]:
dates = pd.date_range('1/1/2010', '12/31/2010')
params = {
    'time_step'    : "60",       
    'start'        : dates[0],
    'stop'         : dates[-1],
    'forcing'      : './input/rc_forcing.nc',     
    'domain'       : './input/rc_domain.nc',
    'state'        : './input/rc_state.nc',
    'forcing_fmt'  : 'netcdf',
    'out_dir'      : './output',
    'out_prefix': 'reynolds',
    'scheduler'    : 'threading',
    'chunks'       : 
        {'lat': 1, 'lon': 1},
    'forcing_vars' : 
        {'prec' : 'prec', 't_max': 't_max', 't_min': 't_min'},
    'state_vars'   : 
        {'prec' : 'prec', 't_max': 't_max', 't_min': 't_min'},
    'domain_vars'  : 
        {'elev': 'elev', 'lat': 'lat', 'lon': 'lon', 'mask': 'mask'}
    }               

ms = MetSim(params)
ms.run()
output = ms.open_output().load()

# Let's look at what we've got
First, let's just look at our output by plotting the shortwave for the year.
We can see a clear annual cycle, as we would hope.

In [ ]:
output['shortwave'].plot()

# Finally, a simple comparison against some observations
While we're at it, let's compare how MetSim simulated shortwave radiation to some observations.
We will just look at some for January. 
First, let's load in the observations and plot the two timeseries.
We see that, generally MetSim has the right timing, but is occasionally off on magnitude.
To look at this a little bit further we also show a scatter plot comparing these values.
Again, we see that MetSim and the observations are fairly well correlated, although there is a decent amount of spread.

In [ ]:
df = pd.read_csv("./reynolds_creek_data/solar_rad_jan_2010.csv", skiprows=[0,1])
df.head()
df.index = pd.DatetimeIndex(df['Date'] + " " + df['Time'])

In [ ]:
sliced = output.sel(time=slice('01/01/2010', '01/31/2010')).isel(lat=0, lon=0, drop=True)
sliced['shortwave'].plot()
df['SRADV.H-1 (watt) '].plot(marker = 'o', ls=':')

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(df['SRADV.H-1 (watt) '], sliced['shortwave'].sel(time=df.index))
ax.set_xlabel(r'Observed shortwave $(W/m^2)$')
ax.set_xlim([0,500])
ax.set_ylabel(r'MetSim shortwave $(W/m^2)$')
ax.set_ylim([0,500])
ax.plot([0, 500], [0, 500], color='k', linestyle='--')